# First RDD

Importing SparkConf (configuration) and SparkContext

In [1]:
from pyspark import SparkConf, SparkContext

Defining a configuration as conf

In [2]:
conf = SparkConf().setAppName("RDD_TRANSFORMATIONS_1")

Creating a spark context sc and passing the configuration created above

In [3]:
sc = SparkContext.getOrCreate(conf=conf)

Taking a simple data collection (List) and creating a rdd by parallelizing the list. After that applying take() action

In [4]:
data = [1, 2, 3, 4, 5]
rdd = sc.parallelize(data)
rdd.take(5)

[1, 2, 3, 4, 5]

# Reading from files

In [5]:
rdd1 = sc.textFile("file:///D:/Code/big-data-stack/pyspark-rdd-operations/data/sample.txt")
rdd1.collect()

['Hello from pySpark']

# Persistance and Caching

### RDD Persistance

In [6]:
oddNums = sc.parallelize(range(1, 1000, 2))
oddNums.take(10)
oddNums.persist()

PythonRDD[7] at RDD at PythonRDD.scala:53

### RDD Caching

In [7]:
rdd2 = sc.textFile("file:///D:/Code/big-data-stack/pyspark-rdd-operations/data/sample.txt")
rdd2.cache()

file:///D:/Code/big-data-stack/pyspark-rdd-operations/data/sample.txt MapPartitionsRDD[9] at textFile at NativeMethodAccessorImpl.java:0

# Spark Cache vs Persist

Using <strong>cache()</strong> and <strong>persist()</strong> methods, Spark provides an optimization mechanism to store the intermediate computation of an RDD, DataFrame, and Dataset so they can be reused in subsequent actions(reusing the RDD, Dataframe, and Dataset computation result’s).

Both caching and persisting are used to save the Spark RDD, Dataframe and Dataset’s. But, the difference is, <strong>RDD cache()</strong> method default saves it to memory (MEMORY_ONLY) whereas <strong>persist()</strong> method is used to store it to user-defined storage level.

When you persist a dataset, each node stores it’s partitioned data in memory and reuses them in other actions on that dataset. And Spark’s persisted data on nodes are fault-tolerant meaning if any partition of a Dataset is lost, it will automatically be recomputed using the original transformations that created it.

# Operations of RDDs

There are two type of data operations we can perform on an RDD, <strong>transformations</strong> and <strong>actions</strong>
- <strong>Transformations</strong> : will return a new RDD as RDDs are generally <strong>Immutable</strong>.
- <strong>Actions</strong> : will return a value

## Transformations on RDDs

Transformations are lazy operations on RDD that create one or more new RDDs.

RDD transformations return a pointer to the new RDD and allow use to create dependencies between RDDs. Each RDD in dependency chain has a function to calculate data and a pointer to the parent RDD.

Spark is lazy, so nothing will be executed until we call a transformation or an action that will trigger the job creation and execution.

## Map Transformation

Passes each element through a function

In [8]:
x = sc.parallelize(["apple", "grapes", "banana", "orange", "kiwi"])
y = x.map(lambda x: (x, 1))
y.collect()

[('apple', 1), ('grapes', 1), ('banana', 1), ('orange', 1), ('kiwi', 1)]

## FlatMap Transformation

Its similar to map transformation, but here each item can be mapped with 0 or more output items, so the function should return a sequence rather than a single item

In [9]:
rdd3 = sc.parallelize([2, 3, 4])
result = rdd3.flatMap(lambda x: range(1, x))
result.collect()

[1, 1, 2, 1, 2, 3]

In [10]:
rdd3 = sc.parallelize([2, 3, 4])
result = rdd3.map(lambda x: range(1, x))
result.collect()

[range(1, 2), range(1, 3), range(1, 4)]

## Filter Transformation

Filtering rows based on some condition

In [11]:
rdd4 = sc.parallelize(range(1, 6))
rdd4.filter(lambda x: x%2 == 0).collect()

[2, 4]

## Sample Transformation

In [12]:
rdd5 = sc.parallelize(range(1, 101))
rdd5.sample(True, .2, seed=42).collect()

[16, 19, 23, 25, 28, 28, 35, 37, 47, 56, 64, 66, 77, 87, 94, 95, 100]

In [13]:
rdd6 = sc.parallelize(range(1, 101))
rdd6.sample(False, .2, seed=42).collect()

[1,
 4,
 5,
 11,
 15,
 18,
 19,
 32,
 33,
 34,
 35,
 36,
 38,
 42,
 43,
 45,
 46,
 47,
 49,
 53,
 57,
 67,
 79,
 80,
 82,
 85,
 87,
 93,
 95,
 99]

## Union Transformation

Combines all the elements of the both rdds

In [14]:
rdd7 = sc.parallelize(range(1, 10))
rdd7_un = sc.parallelize(range(5, 10))
rdd7.union(rdd7_un).collect()

[1, 2, 3, 4, 5, 6, 7, 8, 9, 5, 6, 7, 8, 9]

## Intersection Transformation

Getting the common elements from the rdds.

In [15]:
rdd8 = sc.parallelize(range(1, 10))
rdd8_inter = sc.parallelize(range(5, 10))
sorted(rdd8.intersection(rdd8_inter).collect()) # we are using sorted() to get sorted array in python

[5, 6, 7, 8, 9]

## Distinct Transformation

creates new rdd having distinct elements.

In [16]:
rdd9 = sc.parallelize(range(1, 10))
rdd9_un = sc.parallelize(range(5, 10))
rdd9.union(rdd9_un).collect()

[1, 2, 3, 4, 5, 6, 7, 8, 9, 5, 6, 7, 8, 9]

In [17]:
rdd9 = sc.parallelize(range(1, 10))
rdd9_un = sc.parallelize(range(5, 10))
rdd9.union(rdd9_un).distinct().collect() # gets the distinct from the union

[8, 1, 9, 2, 3, 4, 5, 6, 7]

## SortBy Transformation
returns a sorted rdd.

- True sorts by ascending order.
- False sorts by descending order.

In [18]:
rdd10 = sc.parallelize([1, 7, 10, 3, 2, 20])
rdd10.sortBy(lambda c:c, True).collect()

[1, 2, 3, 7, 10, 20]

## MapPartitions Transformation

Similar to map() transformation, i.e. applies function to each row / record

### map() vs mapPartition()

- <strong>map()</strong> : Spark ```map()``` transformation applies a function to each row in a DataFrame/Dataset and returns the new transformed Dataset.
- <strong>mapPartitions()</strong> : This is exactly the same as map(); the difference being, Spark mapPartitions() provides a facility to do heavy initializations (for example Database connection) once for each partition instead of doing it on every DataFrame row. This helps the performance of the job when you dealing with heavy-weighted initialization on larger datasets.

### PS (important)

Parallelize Syntax:
```
parallelize([collection], number_of_partitions)
```

In [4]:
rdd11 = sc.parallelize([1, 2, 3, 4], 1)
def f(iterator):
    yield sum(iterator)
    
rdd11.mapPartitions(f).collect()

[10]

In [5]:
rdd11.getNumPartitions()

1

## MapPartitionsWithIndex

applies function to all the partitions in the rdd but also maintains indexes for the partitions

In [9]:
rdd12 = sc.parallelize([1, 2, 3, 4])
def f(index, iterator): yield index
rdd12.mapPartitionsWithIndex(f).sum() ## sum of the index

6

## GroupBy

returns new rdd by grouping objects in existing rdd using given groupby key

In [12]:
rdd13 = sc.parallelize(range(1, 11))
results = rdd13.groupBy(lambda x: x%2).collect()

In [20]:
sorted([(x, sorted(y)) for (x, y) in results])

[(0, [2, 4, 6, 8, 10]), (1, [1, 3, 5, 7, 9])]

In [21]:
[(x, y) for (x, y) in results]

[(0, <pyspark.resultiterable.ResultIterable at 0x25c4529b280>),
 (1, <pyspark.resultiterable.ResultIterable at 0x25c452398b0>)]

Here 0, 1 are the indexes of the two groups formed by groupby

In [23]:
[(sorted(y)) for (x, y) in results]

[[2, 4, 6, 8, 10], [1, 3, 5, 7, 9]]

## Zip Operation

In [26]:
rdd14 = sc.parallelize(range(1, 10))
rdd14_new = sc.parallelize(range(20, 30))

result = sc.parallelize(zip(range(1, 10), range(20, 40)))

In [28]:
result.collect()

[(1, 20),
 (2, 21),
 (3, 22),
 (4, 23),
 (5, 24),
 (6, 25),
 (7, 26),
 (8, 27),
 (9, 28)]

In [30]:
sc.parallelize(['a','b','c','d'], 1).zipWithIndex().collect()

[('a', 0), ('b', 1), ('c', 2), ('d', 3)]

In [31]:
sc.parallelize(['a','b','c','d'], 3).zipWithIndex().collect()

[('a', 0), ('b', 1), ('c', 2), ('d', 3)]

## Repartition Transformation

- used to increase or decrease partitions in an RDD.
- does a full shuffle and creates new partition with the data that is distributed evenly.

```
glom() flattens elements on the same partition
```

In [33]:
rdd15 = sc.parallelize(range(1, 20), 4)
rdd15.collect()

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]

In [34]:
rdd15.glom().collect()

[[1, 2, 3, 4], [5, 6, 7, 8, 9], [10, 11, 12, 13, 14], [15, 16, 17, 18, 19]]

In [35]:
sorted(rdd15.glom().collect())

[[1, 2, 3, 4], [5, 6, 7, 8, 9], [10, 11, 12, 13, 14], [15, 16, 17, 18, 19]]

In [37]:
rdd15.repartition(2).glom().collect()

[[1, 2, 3, 4, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19], [5, 6, 7, 8, 9]]

In [42]:
# Length of partition
print('Initial Partition : ', len(rdd15.glom().collect()))
print('After Repartition : ', len(rdd15.repartition(2).glom().collect()))

Initial Partition :  4
After Repartition :  2


## Coalesce Transformation

used to reduce the number of partitions in an RDD. Usually we used it when working with large dataset

In [47]:
sc.parallelize(range(1, 10), 3).glom().collect()

[[1, 2, 3], [4, 5, 6], [7, 8, 9]]

In [48]:
sc.parallelize(range(1, 10), 3).coalesce(2).glom().collect()

[[1, 2, 3], [4, 5, 6, 7, 8, 9]]

## Coalesce vs Repartition

### Coalesce

- uses the existing partitions to minimize the amount of data that's shuffled.
- results in partitions with different amounts of data (at times with much different sizes).
- Faster

### Repartition

